In [11]:
import json
import pandas as pd
import requests
import os
import csv
import sqlite3
import ast

In [42]:
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
 }

##### Taking File as Input:

In [43]:
source = input()

/Users/averygoldberg/Documents/DS 2002/Salary_Data.csv


In [44]:
sourcetype = input()
# url, csv, json

csv


##### Reading File:

In [45]:
try:
    if sourcetype == 'url':
        query = input('Input query: ')
        query = ast.literal_eval(query) 
        # source: https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary
        # converts dictionary as string to dictionary
        file = requests.request("GET",source,headers = header, params = query)
    elif sourcetype == 'json':
        f = open(source,'r')
        file = json.loads(f.read())
        f.close()
    elif sourcetype == 'csv':
        file = open(source)
        file = csv.reader(file)
        header = []
        header = next(file)
        data = []
        for row in file:
            data.append(row)
        file = data
except (FileNotFoundError):
    print('Try again with new source.')

##### Summary of File:

In [46]:
print(file)

[['1.1', '39343.00'], ['1.3', '46205.00'], ['1.5', '37731.00'], ['2.0', '43525.00'], ['2.2', '39891.00'], ['2.9', '56642.00'], ['3.0', '60150.00'], ['3.2', '54445.00'], ['3.2', '64445.00'], ['3.7', '57189.00'], ['3.9', '63218.00'], ['4.0', '55794.00'], ['4.0', '56957.00'], ['4.1', '57081.00'], ['4.5', '61111.00'], ['4.9', '67938.00'], ['5.1', '66029.00'], ['5.3', '83088.00'], ['5.9', '81363.00'], ['6.0', '93940.00'], ['6.8', '91738.00'], ['7.1', '98273.00'], ['7.9', '101302.00'], ['8.2', '113812.00'], ['8.7', '109431.00'], ['9.0', '105582.00'], ['9.5', '116969.00'], ['9.6', '112635.00'], ['10.3', '122391.00'], ['10.5', '121872.00']]


In [47]:
if sourcetype == 'url':
    json_file = file.json()
    count = 0
    for each in json_file:
        for i in each:
            count += 1
    print('The file called from the API %s with parameters %s has' % (source, query), count,'records.')
elif sourcetype == 'json':
    count = 0
    for each in file:
        count += 1
    print('The JSON file %s has' % source, count, 'records. The file provides data for %s.' % file.keys())
elif sourcetype == 'csv':
    rows = 0
    for each in file:
        cols = len(each)
        rows += 1
    print('Number of rows:',rows,'\nNumber of columns:',cols)

Number of rows: 30 
Number of columns: 2


In [48]:
conv_type = input('Input new file type: ')
# type of file to which file will be converted
# SQL, csv, json

Input new file type: json


In [49]:
if conv_type == 'SQL': # converts API source, json, or csv file to sql database
    if sourcetype == 'json':
        df = pd.DataFrame(file, index = [0]) # makes dataframe from dictionary
        filename = input('Input filename ([name].db): ') # takes input for filename for sql database
        conn = sqlite3.connect(filename)
        cur = conn.cursor()
        df.to_sql(filename, conn) # create sql database
    elif sourcetype == 'csv':
        # if source is csv
        dictjson = {} # initializes dictionary for json file
        for i in range(0,cols):
                dictjson[header[i]] = [] # initializes keys in json dictionary
        for each in file:
            for i in range(0,cols):
                dictjson[header[i]].append(each[i]) # adds values to keys in json dictionary
        file = dictjson
        df = pd.DataFrame(file, index = [0]) # makes dataframe from dictionary
        filename = input('Input filename ([name].db): ') # takes input for filename for sql database
        conn = sqlite3.connect(filename)
        cur = conn.cursor()
        df.to_sql(filename, conn) # create sql database
    elif sourcetype == 'url':
        file = file.json()
        df = pd.DataFrame(file, index = [0]) # makes dataframe from dictionary
        filename = input('Input filename ([name].db): ') # takes input for filename for sql database
        conn = sqlite3.connect(filename)
        cur = conn.cursor()
        df.to_sql(filename, conn) # create sql database
elif conv_type == 'json': # converts API source or csv file to json file
    if sourcetype == 'url':
        file_json = file.json() # if source is an API converts results to json
        api_json = json.dumps(file_json) # stores json locally
    elif sourcetype == 'csv':
        # if source is csv
        dictjson = {} # initializes dictionary for json file
        for i in range(0,cols):
                dictjson[header[i]] = [] # initializes keys in json dictionary
        for each in file:
            for i in range(0,cols):
                dictjson[header[i]].append(each[i]) # adds values to keys in json dictionary
        file = json.dumps(dictjson) # saves dictionary as json file locally
elif conv_type == 'csv': # converts API source or json to csv file
    if sourcetype == 'url':
        file = file.json() # turns called API data to json file
        for each in file.keys():
            keys = each
        nfile = file[keys]['result'] # calls list within called data that stores results
        headers = [] # initializes header
        values = [] # initializes values
        for head in nfile[0].keys():
            headers.append(head) # creates header list
        for val in nfile[0].values():
            values.append(val) # creates values list
        filecsv = []
        filecsv.append(headers)
        filecsv.append(values)
        # creates list of headers list and values list
        filename = input('Input filename: ') # '[name].csv'
        with open(filename, 'w', newline="") as file_csv:
            csvwriter = csv.writer(file_csv) # creates a csvwriter object (source: GitHub python/CSV.ipynb)
            for each in filecsv:
                csvwriter.writerow(each) # writes csv file
    elif sourcetype == 'json':
        headers = [] # initializes header
        values = [] # initializes values
        for head in file.keys():
            headers.append(head) # creates header list
        for val in file.values():
            values.append(val) # creates values list
        filecsv = []
        filecsv.append(headers)
        filecsv.append(values)
        # creates list of headers list and values list
        filename = input('Input filename: ') # '[name].csv'
        with open(filename, 'w', newline="") as file_csv:
            csvwriter = csv.writer(file_csv) # creates a csvwriter object (source: GitHub python/CSV.ipynb)
            for each in filecsv:
                csvwriter.writerow(each) # writes csv file